In [1]:
import os


In [2]:
%pwd

'/Users/vivek/work/GIT/end-to-end-mlflow/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/vivek/work/GIT/end-to-end-mlflow'

### defining the entity

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

### adding the configuration manager


In [6]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf


In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_training_config(self)-> TrainingConfig:
        training = self.config.training
        prepare_base_modle = self.config.prepare_base_model
        params = self.params
        trainng_data = os.path.join(self.config.data_ingestion.unzip_dir, "data")
        create_directories(
            [Path(training.root_dir)]
        )

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_modle.updated_base_model_path),
            training_data=Path(trainng_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGUMETATION,
            params_image_size=params.IMAGE_SIZE 
        )

        return training_config
    

In [8]:
import os 
import urllib.request as req
from zipapp import zipfile
import tensorflow as tf
import time


In [9]:
class Training:
    def __init__(self, config = TrainingConfig):
        self.config = config

    def get_base_model(self):
        """
        used to load the previously saved model
        """
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generate(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.20
        )

        dataflow_kwargs = dict(
            target_size = (list(self.config.params_image_size))[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range = 40,
                horizontal_flip = True,
                width_shift_range = 0.2,
                height_shift_range = 0.2,
                shear_range = 0.2,
                zoom_range = 0.2
            )

        else:
            train_datagenerator = valid_datagenerator
        
        self.train_generator = train_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "training",
            shuffle = True,
            **dataflow_kwargs
        )


    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        self.step_per_epochs = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size
    

        optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
        self.model.compile(
        optimizer=optimizer,
        loss=tf.keras.losses.CategoricalCrossentropy(),
        metrics=["accuracy"]
        )


        self.model.fit(
            self.train_generator,
            epochs = self.config.params_epochs,
            steps_per_epoch = self.step_per_epochs,
            validation_steps = self.validation_steps,
            validation_data = self.valid_generator
        )

        self.save_model(
            path = self.config.trained_model_path,
            model=self.model
        )

### creating the pipeline

In [12]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generate()
    training.train()
except Exception as e:
    raise e

[2024-09-24 22:27:07,094 : INFO : common : yaml file: config/config.yaml loaded successfully]
[2024-09-24 22:27:07,096 : INFO : common : yaml file: params.yaml loaded successfully]
[2024-09-24 22:27:07,096 : INFO : common : created directory at: artifacts]
[2024-09-24 22:27:07,098 : INFO : common : created directory at: artifacts/training]
[2024-09-24 22:27:07,223 : WARNING : saving_utils : Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 96 images belonging to 2 classes.
Found 485 images belonging to 2 classes.
Epoch 1/2
30/30 ━━━━━━━━━━━━━━━━━━━━ 8s 218ms/step - accuracy: 0.7478 - loss: 275.0089 - val_accuracy: 0.7604 - val_loss: 9.7468
Epoch 2/2
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 796us/step - accuracy: 0.8750 - loss: 51.5543
[2024-09-24 22:27:15,777 : WARNING : saving_api : You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file 

2024-09-24 22:27:15.754998: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-09-24 22:27:15.755010: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[StatefulPartitionedCall/SGD/Add/ReadVariableOp/_3]]
2024-09-24 22:27:15.755020: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 6912940328657473609
2024-09-24 22:27:15.755025: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 8152812020325118961
2024-09-24 22:27:15.755027: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 6547513640009787602
2024-09-24 22:27:15.755035: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 